### Consulta com expressão regular

In [1]:
from google.cloud import bigquery
import os  
from os.path import join
import re
import xml.etree.cElementTree as ET
import requests;

base_url = "https://api.stackexchange.com";
exception = "Exception";

repostioryName = "../../Dataset_crashanalysis/Github_issues";
files_folders = [];
current_folder = "";
expression = re.compile('([A-Za-z.]+(Error|Exception|Iteration)([^\n]*))');
add = ET.Element("add");

### Obtém diretorios

In [2]:
folders = [name for name in os.listdir(repostioryName)];
folders= sorted(folders, key=str.lower, reverse=True);
def get_next_folder():
    return folders.pop();

### Obtém arquivos

In [3]:
def get_next_file():
    global files_folders;
    global current_folder;
    
    if len(files_folders) <= 0:
        current_folder = get_next_folder();
        files_folders = [name for name in os.listdir(join(repostioryName, current_folder))];
        files_folders = sorted(files_folders, key=str.lower, reverse=True);
    
    if not files_folders:
        return None;
    
    return join(repostioryName, current_folder, files_folders.pop());

### Adiciona elemento ao xml

In [4]:
def add_element(request, response):
    doc = ET.SubElement(add, "doc");
    ET.SubElement(doc, "field", name="Exception_Parse").text = request;
    ET.SubElement(doc, "field", name="Post_link").text = response['link'];
    ET.SubElement(doc, "field", name="View_Count").text = str(response['view_count']);
    ET.SubElement(doc, "field", name="Score").text = str(response['score']);

### Busca das respostas

In [5]:
def search(exeptions):
    for exeption in exeptions:
        url = f"https://api.stackexchange.com/2.3/search/advanced?order=desc&sort=votes&q={exeption}&site=stackoverflow"
        response = requests.get(url);
        data = response.json();
        if not 'error_id' in data:
            items = [k for k in data['items'] if ('java' or 'android' in k['tags']) and (k['is_answered'] == True)];
            items = items[:5] if len(items) > 5 else items;
            for item in items:
                add_element(exeption, item);

### Trata com expressão regular

In [6]:
fileName = get_next_file();
count = 0;
while fileName:
    f = open(fileName, "r")
    text = f.read();
    result = expression.findall(text);
    exeptions = [re.sub('\'.*?\' ','',item[0], flags=re.DOTALL) for item in result];
    search(exeptions);
    fileName = get_next_file();
    count += 1;
    print(count);

1
2


KeyboardInterrupt: 

### Salva XML

In [ ]:
tree = ET.ElementTree(add);
tree.write("../output.xml");